In [1]:
import intake
import satstac
import yaml
import inspect
import intake_xarray
import intake_stac
intake_stac.__version__

'0.3.0.post58'

# Load an Intake Catalog

In [8]:
url = 'https://raw.githubusercontent.com/radiantearth/stac-spec/master/examples/catalog.json'
stac_cat = intake.open_stac_catalog(url)

l1 = stac_cat
l2 = stac_cat['extensions-collection']
l3 = stac_cat['extensions-collection']['proj-example']

print(type(l1))
print(type(l2))
print(type(l3))

<class 'intake_stac.catalog.StacCatalog'>
<class 'intake_stac.catalog.StacCollection'>
<class 'intake_stac.catalog.StacItem'>


In [9]:
b1 = stac_cat['extensions-collection']['proj-example']['B1']
b8 = stac_cat['extensions-collection']['proj-example']['B8']

print(type(b1))
#print(b1)

<class 'intake_xarray.raster.RasterIOSource'>


# Serialize a Single Asset as a .YAML file - Done

In [4]:
b1data = b1._yaml() # For datasources the existing yaml implimentation works.
b1data['metadata'] = {'version':1}

## Confirm that I can save and load this yaml
with open('b1.yaml', 'w') as f:
    f.write(yaml.dump(b1data))

b1cat = intake.open_catalog('b1.yaml')
#list(b1cat)
#b1cat['stac_asset'] #Works!

In [6]:
b8data = b8._yaml() # For datasources the existing yaml implimentation works.
b8data['metadata'] = {'version':1}

## Confirm that I can save and load this yaml
with open('b8.yaml', 'w') as f:
    f.write(yaml.dump(b8data))

b8cat = intake.open_catalog('b8.yaml')
#list(b8cat)
#b8cat['stac_asset'] #Works!

# Serialize 2 Assets as a .YAML file - Done

In [ ]:
## Try to get both B1 and B8 in same YAML in an automated way

l3data = {'metadata':{'version':1}, 'sources':{}}
l3data['metadata'].update(l3.metadata)

for key, source in l3.items():
    l3data['sources'][key] = source._yaml()['sources']['stac_asset']

with open('l3.yaml', 'w') as f:
    f.write(yaml.dump(l3data))

l3catalog = intake.open_catalog('l3.yaml')
#list(l3catalog)
l3catalog

# Testing

Code from the tests

```python
import pytest

class TestDrivers:
    def test_cat_item_yaml(self, pystac_item):
        cat_str = StacItem(pystac_item).yaml()
        d = yaml.load(cat_str)

        for key in ['bbox','date','datetime','geometry','version']:
            assert key in d['metadata']
        for key in ['B02','B03']:
            assert key in d['sources']

    def test_cat_item_yaml_roundtrip(self, pystac_item, tmp_path):
        cat1 = StacItem(pystac_item)
        cat_str = cat1.yaml()

        temp_file = tmp_path/'temp.yaml'
        with open(temp_file, 'w') as f:
            f.write(cat_str)

        cat2 = intake.open_catalog(temp_file)

        for key in ['B02','B03']:
            assert key in cat2
            
        assert cat1.walk() == cat2.walk(), print(cat1) # This one fails!
```

In [ ]:
#print(l3.walk()) # The original StackObject (direct_access: True)
#print(l3catalog.walk()) # The new StackObject from our generated YAML (direct_access: forbid)

In [127]:
#l3.walk()['B1'].describe()['direct_access'] # True
#l3catalog.walk()['B1'].describe()['direct_access'] # 'forbid'
b1.describe()['direct_access'] # True

True